# Подготовка

In [ ]:
!pip install google
!pip install python-dotenv
!pip install pyowm
!pip install scikit-learn

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

In [ ]:
def failure(*args: tuple):
  return "Не удалось распознать команду"

In [ ]:
config = {
    "intents": {
        "greeting": {
            "examples": ["привет", "здравствуй", "добрый день",
                         "hello", "good morning"],
            "responses": func
        },
        "farewell": {+ 
            "examples": ["пока", "до свидания", "увидимся"],
            "responses": func
        },
        "meetings_search": {
            "examples": ["ближайшие встречи",
                         "какие встречи",
                         "назначенные встречи"],
            "responses": search_meeting
        },
        "meetings_result": {
            "examples": ["Как прошла встреча",
                         "Результат встречи"],
            "responses": search_meeting_result
        },
        "meeting_record_result": {
            "examples": ["В результате",
                         "Результат встречи",
                         ""],
            "responses": search_meeting_result
        },
        "task_add": {
            "examples": ["Запиши",
                         "Добавь задачу"],
            "responses": search_meeting_result
        },
        

    },

    "failure_phrases": failure
}

In [ ]:
def prepare_corpus():
    corpus = []
    target_vector = []
    for intent_name, intent_data in config["intents"].items():
        for example in intent_data["examples"]:
            corpus.append(example)
            target_vector.append(intent_name)

    training_vector = vectorizer.fit_transform(corpus)
    classifier_probability.fit(training_vector, target_vector)
    classifier.fit(training_vector, target_vector)
vectorizer = TfidfVectorizer(analyzer="char", ngram_range=(2, 3))
classifier_probability = LogisticRegression()
classifier = LinearSVC()
prepare_corpus()

In [ ]:
def get_intent(request):
    """
    Получение наиболее вероятного намерения в зависимости от запроса пользователя
    :param request: запрос пользователя
    :return: наиболее вероятное намерение
    """
    best_intent = classifier.predict(vectorizer.transform([request]))[0]

    index_of_best_intent = list(classifier_probability.classes_).index(best_intent)
    probabilities = classifier_probability.predict_proba(vectorizer.transform([request]))[0]

    best_intent_probability = probabilities[index_of_best_intent]

    print(best_intent_probability)
    if best_intent_probability > 0.15:
        return best_intent

In [ ]:
get_intent("Мои назd наченные встречи")

0.3217671427833537


'meetings_search'

In [ ]:
def get_answer(request):

    voice_input_parts = request.split(" ")

    if len(voice_input_parts) == 1:
        intent = get_intent(voice_input)
        if intent:
            return config["intents"][intent]["responses"]()
        else:
            return config["failure_phrases"]()

    if len(voice_input_parts) > 1:
        for guess in range(2, len(voice_input_parts)):
            intent = get_intent((" ".join(voice_input_parts[0:guess])).strip())
            print(intent)
            if intent:
                command_options = [voice_input_parts[guess:len(voice_input_parts)]]
                print(command_options)
                return config["intents"][intent]["responses"](*command_options)
                break
        return config["failure_phrases"]()

In [ ]:
get_answer("Какие Назначенные встречи")

0.3126867454554005
meetings_search
[['встречи']]
Looking for a meeting...


'У Вас назначена Встреча с представителями Сбера на 12.11.2021 в 12:00'

# База данных

## Задачи

In [ ]:
tasks = [
         {
            "name": "Разработать дизайн приложения",
            "status": "Работа над календарем",
            "team": [
                     {
                         "name": "Александра Зверева",
                         "icon": "",
                         "link": "t.me/zvendra"
                     }
            ]

         }
]

## Календарь

In [ ]:
events = [
          {
              "type": "event",
              "name": "Встреча с Павлом Ильином",
              "date": "28.11.2021",
              "timestart": "14:00",
              "timeend": "16:00"
          },
          {
              "type": "event",
              "date": "28.11.2021",
              "timestart": "18:00",
              "timeend": "22:00",
              "name": "Хакатон T1",
              "result": ""
          },
          {
              "type": "meeting",
              "date": "12.11.2021",
              "time": "12:00",
              "name": "Встреча с представителями Сбера",
              "result": "Подписан договор о сотрудничестве в сфере ИИ"
          }

]

# Функции ассистента

In [ ]:
from datetime import datetime, date
def search_meeting(*args: tuple):
    print("Looking for a meeting...")
    for event in events:
        if event['type'] == "meeting":
            datelist = events[0]["date"].split(".")
            timestart = events[0]["timestart"].split(":")
            meetingdate = datetime(int(datelist[2]),int(datelist[1]),int(datelist[0]), int(timestart[0]), int(timestart[1]))
            if meetingdate > datetime.now():
                return "У Вас назначена " + event["name"] + " на " + event["date"] + " в " + event["time"]

In [ ]:
def search_meeting_result(*args: tuple):
    print("Looking for a result...")
    for event in events:
        if event['type'] == "meeting":
            datelist = events[0]["date"].split(".")
            timestart = events[0]["timestart"].split(":")
            meetingdate = datetime(int(datelist[2]),int(datelist[1]),int(datelist[0]), int(timestart[0]), int(timestart[1]))
            if meetingdate > datetime.now():
                return "У Вас назначена " + event["name"] + " на " + event["date"] + " в " + event["time"]

In [ ]:
def record_meeting_result(*args: tuple):
    

In [ ]:
from datetime import datetime, date
datelist = events[0]["date"].split(".")
timestart = events[0]["timestart"].split(":")
date1 = datetime(int(datelist[2]),int(datelist[1]),int(datelist[0]), int(timestart[0]), int(timestart[1]))

# DeepPavlov integration

In [1]:
!pip install deeppavlov

In [53]:
from deeppavlov import configs, build_model

ner_model = build_model(configs.ner.ner_ontonotes_bert_mult_torch, download=True)

/usr/local/lib/python3.7/dist-packages/cryptography/hazmat/backends/openssl/x509.py:17: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  utils.DeprecatedIn35,
2021-11-27 12:36:09.882 INFO in 'deeppavlov.core.data.utils'['utils'] at line 95: Downloading from http://files.deeppavlov.ai/v1/ner/ner_ontonotes_bert_mult_torch.tar.gz to /root/.deeppavlov/ner_ontonotes_bert_mult_torch.tar.gz
100%|██████████| 1.38G/1.38G [01:13<00:00, 18.7MB/s]
2021-11-27 12:37:24.428 INFO in 'deeppavlov.core.data.utils'['utils'] at line 272: Extracting /root/.deeppavlov/ner_ontonotes_bert_mult_torch.tar.gz archive into /root/.deeppavlov/models


Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

2021-11-27 12:37:51.351 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/ner_ontonotes_bert_mult_torch/bert-base-multilingual-cased/tag.dict]


Downloading:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at 

In [16]:
ner_model(['Добавь задачу отправить письмо Александру Ивановичу в 10 00'])

[[['Добавь',
   'задачу',
   'отправить',
   'письмо',
   'Александру',
   'Ивановичу',
   'в',
   '10',
   '00']],
 [['O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O']]]

In [34]:
!pip3 install git+https://github.com/Koziev/rulemma

  Cloning https://github.com/Koziev/rulemma to /tmp/pip-req-build-8y2ycw_y
  Running command git clone -q https://github.com/Koziev/rulemma /tmp/pip-req-build-8y2ycw_y
  Created wheel for rulemma: filename=rulemma-0.0.44-py3-none-any.whl size=16113694 sha256=2c28d4c673f38823d4ce2996953e52982d6820b4f07bd2365305360790da5eaf
  Stored in directory: /tmp/pip-ephem-wheel-cache-0gt4ycz6/wheels/b7/89/eb/a075486911bb14812212596d44434c565b9d6f03659c09fbbe
Successfully built rulemma


In [38]:
lemmatizer.lemmatize("Александру")

ValueError: ignored

In [67]:
def returndict(inp):
    ans = {}
    nertokens = ner_model([inp])
    print(nertokens)
    for i in range(len(nertokens[0][0])):
        if str(nertokens[1][0][i]).count("PER") > 0:
            if list(ans.keys()).count("person") == 0:
                ans['person'] = nertokens[0][0][i]
            else:
                ans['person'] += " " + nertokens[0][0][i]
        elif str(nertokens[1][0][i]).count("LOC") > 0 or str(nertokens[1][0][i]).count("FAC") > 0 or str(nertokens[1][0][i]).count("GPE") > 0:
            if list(ans.keys()).count("location") == 0:
                ans['location'] = nertokens[0][0][i]
            else:
                ans['location'] += " " + nertokens[0][0][i]
        elif str(nertokens[1][0][i]).count("ORG") > 0:
            if list(ans.keys()).count("organisation") == 0:
                ans['organisation'] = nertokens[0][0][i]
            else:
                ans['organisation'] += " " + nertokens[0][0][i]
        elif str(nertokens[1][0][i]).count("DATE") > 0:
            if list(ans.keys()).count("time") == 0:
                ans['time'] = nertokens[0][0][i]
            else:
                ans['time'] += " " + nertokens[0][0][i]
        elif str(nertokens[1][0][i]).count("MONEY") > 0:
            if list(ans.keys()).count("money") == 0:
                ans['money'] = nertokens[0][0][i]
            else:
                ans['money'] += " " + nertokens[0][0][i]
        elif str(nertokens[1][0][i]).count("GPE") > 0:
            if list(ans.keys()).count("money") == 0:
                ans['money'] = nertokens[0][0][i]
            else:
                ans['money'] += " " + nertokens[0][0][i]
        elif str(nertokens[1][0][i]).count("MONEY") > 0:
            if list(ans.keys()).count("money") == 0:
                ans['money'] = nertokens[0][0][i]
            else:
                ans['money'] += " " + nertokens[0][0][i]
        elif str(nertokens[1][0][i]).count("EVENT") > 0:
            if list(ans.keys()).count("event") == 0:
                ans['event'] = nertokens[0][0][i]
            else:
                ans['event'] += " " + nertokens[0][0][i]
        
    return ans

In [69]:
returndict("В результате встречи была установлена цена 500 рублей за один смартфон, подписан договор о проведении конференции AIContest")

[[['В', 'результате', 'встречи', 'была', 'установлена', 'цена', '500', 'рублей', 'за', 'один', 'смартфон', ',', 'подписан', 'договор', 'о', 'проведении', 'конференции', 'AIContest']], [['O', 'O', 'O', 'O', 'O', 'O', 'B-MONEY', 'I-MONEY', 'O', 'B-CARDINAL', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG']]]


{'money': '500 рублей', 'organisation': 'AIContest'}

In [ ]:
def get_answer(request):

    voice_input_parts = request.split(" ")

    if len(voice_input_parts) == 1:
        intent = get_intent(voice_input)
        if intent:
            return config["intents"][intent]["responses"]()
        else:
            return config["failure_phrases"]()

    if len(voice_input_parts) > 1:
        for guess in range(2, len(voice_input_parts)):
            intent = get_intent((" ".join(voice_input_parts[0:guess])).strip())
            print(intent)
            if intent:
                command_options = [voice_input_parts[guess:len(voice_input_parts)]]
                print(command_options)
                return config["intents"][intent]["responses"](*command_options)
                break
        return config["failure_phrases"]()